<a href="https://colab.research.google.com/github/Befezdow/ai_test/blob/master/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
%matplotlib inline

drive.mount('/content/gdrive')
names_array = ['gender', 'age', 'class1', 'class2', 'class3', *['t{}'.format(i + 1) for i in range(60000)]]
url = '/content/gdrive/My Drive/ML/half.csv'
data = pd.read_csv(url, names=names_array)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Подготавливаем данные, заполняя пропущенные значения медианой.

In [2]:
data = data.fillna(data.median(axis = 0), axis = 0)
data

,gender,age,class1,class2,class3,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,...,t59961,t59962,t59963,t59964,t59965,t59966,t59967,t59968,t59969,t59970,t59971,t59972,t59973,t59974,t59975,t59976,t59977,t59978,t59979,t59980,t59981,t59982,t59983,t59984,t59985,t59986,t59987,t59988,t59989,t59990,t59991,t59992,t59993,t59994,t59995,t59996,t59997,t59998,t59999,t60000
0,Female,85,3,5.0,NaN,0.003242,0.012132,0.022637,0.010953,-0.024891,-0.032752,-0.031432,-0.042890,-0.048659,-0.037955,-0.026816,-0.017216,-0.014815,-0.024370,-0.027953,-0.037105,-0.046662,-0.052641,-0.054785,-0.054178,-0.034856,-0.023726,-0.036748,-0.031842,-0.021913,-0.038332,-0.045422,-0.044497,-0.042550,-0.038772,-0.043089,-0.055117,-0.062105,-0.062300,-0.062635,...,-0.026842,-0.028914,-0.028548,-0.029272,-0.031896,-0.036031,-0.039794,-0.033593,-0.028113,-0.031301,-0.041995,-0.045999,-0.043968,-0.039291,-0.030942,-0.031698,-0.030754,-0.033479,-0.033446,-0.036413,-0.038551,-0.035920,-0.036477,-0.036518,-0.041184,-0.039042,-0.033992,-0.033940,-0.031528,-0.026267,-0.028246,-0.028318,-0.025569,-0.021279,-0.018593,-0.032023,-0.046202,-0.042003,-0.034466,-0.039000
1,Male,79,9,5.0,NaN,0.012054,0.017017,0.015951,0.020941,0.020960,0.021158,0.021051,0.021062,0.021122,0.021278,0.026533,0.026235,0.016037,0.013366,0.013222,0.008356,0.005556,0.007938,0.008593,0.008398,0.008547,0.008369,0.008677,0.006235,0.000651,-0.001857,-0.001712,-0.007047,-0.009004,-0.012014,-0.019256,-0.019027,-0.019518,-0.018919,-0.020128,...,-0.934049,-0.949376,-0.962051,-0.963686,-0.961742,-0.952030,-0.939060,-0.934415,-0.936366,-0.939326,-0.932011,-0.926725,-0.928331,-0.929013,-0.921389,-0.906309,-0.900787,-0.901273,-0.895911,-0.893740,-0.891627,-0.885869,-0.881577,-0.873489,-0.866113,-0.865862,-0.859139,-0.852907,-0.843380,-0.833925,-0.833698,-0.836701,-0.827418,-0.827810,-0.816231,-0.819793,-0.797862,-0.819597,-0.771195,-0.845490
2,Male,66,3,5.0,NaN,-0.009508,-0.008508,-0.009508,-0.008508,-0.007508,-0.006508,-0.006508,-0.006508,-0.005508,-0.006508,-0.005508,-0.004508,-0.003508,-0.004508,-0.004508,-0.004508,-0.005508,-0.004508,-0.003508,-0.004508,-0.003508,-0.002508,-0.000508,-0.001508,-0.000508,-0.000508,-0.001508,-0.001508,-0.001508,-0.000508,0.000492,0.002492,0.005492,0.006492,0.007492,...,-0.051053,-0.052053,-0.051053,-0.049053,-0.047053,-0.044053,-0.040053,-0.037053,-0.036053,-0.034053,-0.034053,-0.034053,-0.033053,-0.031053,-0.028053,-0.025053,-0.022053,-0.019053,-0.017053,-0.017053,-0.016053,-0.016053,-0.014053,-0.013053,-0.010053,-0.007053,-0.003053,0.001947,0.004947,0.007947,0.009947,0.011947,0.012947,0.015947,0.018947,0.023947,0.027947,0.030947,0.034947,0.037947
3,Male,93,2,5.0,NaN,-0.002271,0.004177,-0.002077,-0.005875,-0.004685,-0.008117,-0.019722,-0.036185,-0.037616,-0.025323,-0.023199,-0.035744,-0.043484,-0.030469,-0.022611,-0.010540,0.004805,0.020337,0.029668,0.017382,0.004737,0.002755,0.014075,0.015677,0.002024,-0.002544,0.000068,0.000271,0.011654,0.022805,0.028631,0.029834,0.030584,0.035327,0.051485,...,-0.117502,-0.208445,-0.254530,-0.270954,-0.254278,-0.223832,-0.205055,-0.179093,-0.157585,-0.127692,-0.089728,-0.074150,-0.056856,-0.038932,-0.024184,-0.016817,-0.014031,-0.011765,-0.022771,-0.023586,-0.013769,-0.004949,-0.003625,-0.013567,-0.019471,-0.015372,-0.021787,-0.028885,-0.028997,-0.021737,-0.013160,-0.006182,-0.003825,-0.002177,-0.001466,-0.006676,-0.011236,-0.016540,-0.016289,-0.022542
4,Male,65,2,5.0,NaN,-0.030600,-0.027425,-0.013923,-0.023743,-0.024572,-0.036078,-0.034740,-0.013670,-0.009571,-0.022441,-0.033034,-0.032663,-0.018181,-0.017317,-0.030576,-0.032829,-0.010305,0.005287,0.006696,-0.002352,-0.005559,-0.005361,-0.008254,0.002175,0.007531,0.002662,0.000376,-0.007697,-0.016977,-0.009894,-0.000388,-0.012127,-0.007840,-0.002470,-0.005233,...,1.016910,0.710028,0.406775,0.104651,-0.181270,-0.432753,-0.645005,-0.816434,-0.952654,-1.038151,-1.073172,-1.060904,-1.006902,-0.914312,-0.803500,-0.697

Осуществляем аугментацию данных. На каждый датчик идет по 5000 записей. Разбиваем так, чтобы на каждый приходилось по 250, тем самым размножая данные.

In [3]:
count_of_iterations = 20
abductions_count = 12
needed_per_abduction = 250

non_sliceable_columns = data.iloc[:,0:5]
sliceable_columns = data.iloc[:,5:]
sliceable_columns_count = sliceable_columns.shape[1]
per_abduction = int(sliceable_columns_count / abductions_count)

new_data = pd.DataFrame()
for i in range(count_of_iterations):
  random_position = random.randint(0, per_abduction - needed_per_abduction)

  total_columns_slice = pd.DataFrame()
  for j in range(abductions_count):
    start_column = j * per_abduction + random_position
    end_column = j * per_abduction + random_position + needed_per_abduction
    columns_slice = sliceable_columns.iloc[:, start_column : end_column]
    total_columns_slice = pd.concat([total_columns_slice, columns_slice.copy()], axis=1, sort=False, ignore_index=True)
  
  completed_slice = pd.concat([non_sliceable_columns.copy(), total_columns_slice], axis=1, sort=False, ignore_index=True)
  new_data = pd.concat([new_data, completed_slice], axis=0, sort=False, ignore_index=True)

new_data.columns = ['gender', 'age', 'class1', 'class2', 'class3', *['t{}_{}'.format(i + 1, j + 1) for i in range(abductions_count) for j in range(needed_per_abduction)]]
new_data

,gender,age,class1,class2,class3,t1_1,t1_2,t1_3,t1_4,t1_5,t1_6,t1_7,t1_8,t1_9,t1_10,t1_11,t1_12,t1_13,t1_14,t1_15,t1_16,t1_17,t1_18,t1_19,t1_20,t1_21,t1_22,t1_23,t1_24,t1_25,t1_26,t1_27,t1_28,t1_29,t1_30,t1_31,t1_32,t1_33,t1_34,t1_35,...,t12_211,t12_212,t12_213,t12_214,t12_215,t12_216,t12_217,t12_218,t12_219,t12_220,t12_221,t12_222,t12_223,t12_224,t12_225,t12_226,t12_227,t12_228,t12_229,t12_230,t12_231,t12_232,t12_233,t12_234,t12_235,t12_236,t12_237,t12_238,t12_239,t12_240,t12_241,t12_242,t12_243,t12_244,t12_245,t12_246,t12_247,t12_248,t12_249,t12_250
0,Female,85,3,5.0,NaN,0.016549,0.014954,0.012889,0.008263,0.005147,0.008132,0.005443,-0.003760,-0.010286,-0.014781,-0.014339,-0.014399,-0.014566,-0.014746,-0.026694,-0.026782,-0.021214,-0.015065,-0.013878,-0.024817,-0.026581,-0.023111,-0.033577,-0.037041,-0.025178,-0.022258,-0.033493,-0.032090,-0.022201,-0.018973,-0.021674,-0.021628,-0.025337,-0.037026,-0.039719,...,0.052143,0.060409,0.066680,0.067279,0.074291,0.084458,0.089906,0.100135,0.107055,0.107448,0.109035,0.118069,0.122351,0.127153,0.135410,0.142180,0.150917,0.154615,0.162612,0.165786,0.172485,0.181231,0.183015,0.190131,0.195230,0.203338,0.208605,0.208773,0.216209,0.218638,0.223321,0.226572,0.230420,0.238045,0.241385,0.238917,0.238549,0.245322,0.251146,0.256163
1,Male,79,9,5.0,NaN,-0.052429,-0.067265,-0.064600,-0.059559,-0.056273,-0.052329,-0.046943,-0.049499,-0.052027,-0.046685,-0.043933,-0.039450,-0.034727,-0.034300,-0.034480,-0.032268,-0.026292,-0.024784,-0.026646,-0.021922,-0.023923,-0.031216,-0.037237,-0.038934,-0.029302,-0.028793,-0.034961,-0.044006,-0.049391,-0.056515,-0.061653,-0.064441,-0.067687,-0.066668,-0.067727,...,0.031163,0.036018,0.041508,0.043473,0.038735,0.033416,0.030893,0.031122,0.032785,0.034395,0.036293,0.035946,0.036487,0.033597,0.030783,0.031247,0.031010,0.032725,0.036275,0.038975,0.038217,0.033280,0.025583,0.023212,0.018013,0.010379,0.000993,-0.009221,-0.019418,-0.027209,-0.031581,-0.039220,-0.050121,-0.062467,-0.071810,-0.084934,-0.097346,-0.102382,-0.107586,-0.112518
2,Male,66,3,5.0,NaN,-0.016508,-0.016508,-0.018508,-0.018508,-0.019508,-0.021508,-0.021508,-0.022508,-0.022508,-0.022508,-0.020508,-0.018508,-0.018508,-0.019508,-0.018508,-0.019508,-0.020508,-0.019508,-0.019508,-0.020508,-0.020508,-0.019508,-0.018508,-0.018508,-0.020508,-0.018508,-0.018508,-0.017508,-0.016508,-0.016508,-0.015508,-0.014508,-0.013508,-0.011508,-0.012508,...,0.063947,0.059947,0.055947,0.049947,0.044947,0.038947,0.032947,0.026947,0.020947,0.015947,0.009947,0.005947,0.000947,-0.003053,-0.007053,-0.010053,-0.013053,-0.016053,-0.017053,-0.020053,-0.022053,-0.026053,-0.029053,-0.032053,-0.036053,-0.039053,-0.041053,-0.043053,-0.044053,-0.044053,-0.046053,-0.046053,-0.048053,-0.049053,-0.051053,-0.052053,-0.052053,-0.051053,-0.051053,-0.051053
3,Male,93,2,5.0,NaN,-0.006358,-0.004378,-0.010646,-0.021001,-0.017053,-0.010772,-0.010177,-0.018495,-0.027695,-0.023000,-0.023187,-0.022453,-0.025262,-0.022574,-0.015398,-0.015384,-0.017600,-0.029939,-0.033097,-0.032956,-0.025206,-0.019666,-0.013470,-0.010238,-0.014954,-0.017842,-0.028256,-0.035429,-0.033555,-0.032924,-0.032899,-0.032987,-0.024913,-0.015291,-0.023086,...,-0.004369,-0.005758,-0.014131,-0.022217,-0.031163,-0.036386,-0.036761,-0.035011,-0.031678,-0.028751,-0.038725,-0.046730,-0.047370,-0.056964,-0.064841,-0.067037,-0.068521,-0.071661,-0.071796,-0.074266,-0.079654,-0.084220,-0.084422,-0.087172,-0.097393,-0.105102,-0.107502,-0.105510,-0.102293,-0.099112,-0.100404,-0.095375,-0.097467,-0.102970,-0.101719,-0.102762,-0.099614,-0.096544,-0.093858,-0.092545
4,Male,65,2,5.0,NaN,-0.035120,-0.043445,-0.047906,-0.050621,-0.037763,-0.035400,-0.051020,-0.063034,-0.065548,-0.051117,-0.030259,-0.019779,-0.015844,-0.017484,-0.025842,-0.007225,-0.000765,-0.019969,-0.012985,-0.016820,-0.015621,-0.012320,-0.021223,-0.037577,-0.040804,-0.048260,-0.060690,-0.064116,-0.073049,-0.082957,-0.080848,-0.077939,-0.081025,-0.097868,-0.097993,...,0.099079,0.086923,0.078883,0.064235,0.053643,0.046011,0.0

In [0]:
data = new_data

Выкидываем лишние классовые признаки, а затем осуществляем векторизацию и нормализацию данных.

In [5]:
# убираем лишние классовые признаки
data = data.drop('class2', axis = 1)
data = data.drop('class3', axis = 1)
 
#собираем числовые колонки
numerical_columns = [c for c in data.columns if data[c].dtype.name != 'object' and c != 'class1']

# векторизируем колонку gender
data.at[data['gender'] == 'Male', 'gender'] = 0
data.at[data['gender'] == 'Female', 'gender'] = 1

# нормализуем числовые атрибуты
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)
data[numerical_columns] = data_numerical

data

,gender,age,class1,t1_1,t1_2,t1_3,t1_4,t1_5,t1_6,t1_7,t1_8,t1_9,t1_10,t1_11,t1_12,t1_13,t1_14,t1_15,t1_16,t1_17,t1_18,t1_19,t1_20,t1_21,t1_22,t1_23,t1_24,t1_25,t1_26,t1_27,t1_28,t1_29,t1_30,t1_31,t1_32,t1_33,t1_34,t1_35,t1_36,t1_37,...,t12_211,t12_212,t12_213,t12_214,t12_215,t12_216,t12_217,t12_218,t12_219,t12_220,t12_221,t12_222,t12_223,t12_224,t12_225,t12_226,t12_227,t12_228,t12_229,t12_230,t12_231,t12_232,t12_233,t12_234,t12_235,t12_236,t12_237,t12_238,t12_239,t12_240,t12_241,t12_242,t12_243,t12_244,t12_245,t12_246,t12_247,t12_248,t12_249,t12_250
0,1,1.177837,3,0.060619,0.112569,0.098377,0.063351,0.038589,0.059425,0.038508,-0.028753,-0.047100,-0.109465,-0.106200,-0.105574,-0.105408,-0.106798,-0.197116,-0.197339,-0.154869,-0.107349,-0.096989,-0.179673,-0.192244,-0.165445,-0.245799,-0.272981,-0.182565,-0.160230,-0.245117,-0.232969,-0.158719,-0.135263,-0.155965,-0.156344,-0.185488,-0.274556,-0.294439,-0.252817,-0.297558,...,0.170799,0.197233,0.217456,0.218781,0.241943,0.275784,0.294597,0.330330,0.287609,0.355909,0.361345,0.318365,0.329930,0.421958,0.451193,0.475909,0.507545,0.522732,0.553602,0.566332,0.587975,0.616681,0.622026,0.645292,0.660911,0.686160,0.702553,0.702866,0.728653,0.599154,0.755203,0.767296,0.782177,0.808809,0.821780,0.814646,0.813769,0.838785,0.860384,0.879626
1,0,0.808686,9,-0.220945,-0.512235,-0.487402,-0.445893,-0.419258,-0.387990,-0.346460,-0.363912,-0.215545,-0.344582,-0.325701,-0.292725,-0.256789,-0.254341,-0.256136,-0.239113,-0.193664,-0.181841,-0.194907,-0.157471,-0.171843,-0.227535,-0.273761,-0.287421,-0.214022,-0.210019,-0.256261,-0.323095,-0.364098,-0.418475,-0.457435,-0.479467,-0.505267,-0.498509,-0.506252,-0.479992,-0.481780,...,0.100267,0.115356,0.133170,0.139303,0.123334,0.105560,0.097646,0.099511,0.083259,0.111417,0.118124,0.092704,0.093999,0.109061,0.099896,0.101720,0.101460,0.108104,0.120763,0.129778,0.126400,0.108908,0.082017,0.073313,0.054536,0.027418,-0.005192,-0.040089,-0.074756,-0.086222,-0.116168,-0.142789,-0.180749,-0.223433,-0.255734,-0.301006,-0.343999,-0.362135,-0.381297,-0.399656
2,0,0.008860,3,-0.074318,-0.126518,-0.138965,-0.137660,-0.145199,-0.159910,-0.159545,-0.166133,-0.096419,-0.166408,-0.151954,-0.136268,-0.135005,-0.142725,-0.135054,-0.141956,-0.149469,-0.141398,-0.140162,-0.146629,-0.145636,-0.137840,-0.130673,-0.131582,-0.146944,-0.131655,-0.131364,-0.122678,-0.115717,-0.116666,-0.109469,-0.102604,-0.096167,-0.081756,-0.088647,-0.080849,-0.074991,...,0.210481,0.195681,0.181518,0.160916,0.144055,0.124006,0.104500,0.085549,0.050686,0.049677,0.030033,0.010272,-0.003655,-0.013515,-0.027144,-0.037588,-0.047768,-0.057822,-0.061941,-0.073427,-0.080793,-0.094723,-0.105389,-0.116065,-0.130458,-0.141339,-0.148525,-0.155395,-0.158753,-0.133181,-0.165639,-0.166186,-0.173650,-0.177356,-0.184321,-0.187733,-0.187882,-0.184852,-0.185620,-0.186379
3,0,1.670038,2,-0.032886,-0.034337,-0.079530,-0.156382,-0.126897,-0.080463,-0.076281,-0.136725,-0.117354,-0.170038,-0.171823,-0.165739,-0.185716,-0.165857,-0.111479,-0.110556,-0.127248,-0.221349,-0.244374,-0.242091,-0.181696,-0.139055,-0.092188,-0.068485,-0.104585,-0.126585,-0.205368,-0.258223,-0.244481,-0.240503,-0.240613,-0.242071,-0.182288,-0.110341,-0.168650,-0.211122,-0.200644,...,-0.019184,-0.024879,-0.053131,-0.080013,-0.109837,-0.127227,-0.128146,-0.121674,-0.094110,-0.099916,-0.132707,-0.134477,-0.136416,-0.193821,-0.221174,-0.229799,-0.235621,-0.246982,-0.249496,-0.260059,-0.278809,-0.294352,-0.295310,-0.304941,-0.340341,-0.366822,-0.375048,-0.368257,-0.357333,-0.286674,-0.351434,-0.335066,-0.343259,-0.362557,-0.358632,-0.362423,-0.351815,-0.341973,-0.333781,-0.330350
4,0,-0.052666,2,-0.150292,-0.331222,-0.361199,-0.378785,-0.281284,-0.262715,-0.376423,-0.463092,-0.270106,-0.377240,-0.224276,-0.145768,-0.115001,-0.127455,-0.190651,-0.048436,0.001390,-0.144936,-0.090141,-0.118346,-0.108130,-0.082778,-0.151417,-0.277068,-0.301753,-0.358345,-0.451578,-0.475204,-0.542801,-0.617950,-0.602185,-0.581346,-0.605977,-0.734230,-0.735142,-0.70202

Разбиваем данные на входы и выходы.

In [0]:
# бьем данные на входы и выходы
x = data.drop('class1', axis = 1)
y = data['class1']

Разбиваем данные на обучающую и тестовую выборки.

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

n_train, _ = x_train.shape 
n_test,  _ = x_test.shape 

print('Train dataset size: {}'.format(n_train))
print('Test dataset size: {}'.format(n_test))

Train dataset size: 17500
Test dataset size: 7500


# KMeans

In [0]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=len(new_data['class1'].unique()), random_state=42).fit(x)

In [9]:
kmeans_cluster = kmeans.predict(x) + 1
print(np.unique(kmeans.labels_))
check = kmeans_cluster == y
np.where(check == True)[0].shape

[0 1 2 3 4 5 6 7 8]


(1522,)

Количество классов совпадает, т.к. модель подразумевает передачу их количества. Однако стоит заметить, что пересечение с реальным разбиением очень слабое (1522 из 25000).

# DBSCAN

In [10]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN().fit(x)
np.unique(dbscan.labels_)

array([-1])

DBSCAN дает разбиение всего на 1 класс, что является не приемлемым, так как изначальные данные бьются на 9 классов.

# AffinityPropagation

In [1]:
from sklearn.cluster import AffinityPropagation

affinity = AffinityPropagation().fit(x)
np.unique(affinity.labels_)

NameError: ignored

Не получилось проверить данную модель, так как она не помещается в оперативную память (более 13 гб).

# Spectral Clustering
 

In [0]:
from sklearn.cluster import SpectralClustering

spectral = SpectralClustering(n_clusters=len(new_data['class1'].unique()), random_state=42).fit(x)
spectral_cluster = spectral.predict(x) + 1
print(np.unique(spectral.labels_))
check_spectral = spectral_cluster == y
np.where(check_spectral == True)[0].shape

Также не получилось проверить данную модель, так как она не помещается в оперативную память (более 13 гб).

# Mean-shift

In [11]:
from sklearn.cluster import MeanShift, estimate_bandwidth

bandwidth = estimate_bandwidth(x, quantile=0.2, n_samples=1000)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(x)
ms_cluster = ms.predict(x) + 1
print(np.unique(ms.labels_))
check_ms = ms_cluster == y
np.where(check_ms == True)[0].shape

[0]


(3340,)

Mean-shift, также как и DBSCAN, дает разбиение всего на 1 класс, что является не приемлемым, так как изначальные данные бьются на 9 классов. Пересечение с реальным разбиением также слабое (3340 из 25000).

Исходя из результатов, можно сделать вывод что данные плохо поддаются кластеризации. Модели, которые не запрашивают кол-во кластеров, не могут разделить данные и в итоге получается всего 1 кластер. KMeans же принимает кол-во классов, но дает очень плохую точность (менее 20%).